In [25]:
import psycopg2
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
connection = psycopg2.connect(
    host = '152.70.249.200',
    database = 'kdsg_sooldamwha',
    user = 'kdsg',
    password = 'kdsg1234')
cursor = connection.cursor()
cursor.execute("SELECT ID, PRODUCT_ID, PRODUCT_OPTION_ID FROM ORDERS")
result = cursor.fetchall()
# connection.commit()

In [22]:
df_order = pd.DataFrame(result, columns = ['ID', 'PRODUCT_ID', 'PRODUCT_OPTION_ID'])

In [24]:
df_order.describe()

,ID,PRODUCT_ID,PRODUCT_OPTION_ID
count,154188.000000,154188.000000,154188.000000
mean,161905.904532,475.830849,771.698083
std,59147.185563,437.126129,549.766571
min,146.000000,40.000000,66.000000
25%,130485.750000,156.000000,273.000000
50%,169549.500000,244.000000,525.000000
75%,208642.250000,1061.000000,1301.000000
max,247936.000000,1270.000000,1663.000000


# productid와 product optioin id를 합쳐서 V노드로 생성 필요 
product 자리수: 0000 네자리수로 표현
product_option_id자리수 : 0000네자리수로 표현

> 신규 변수 생성 po_id 


In [29]:
df_order['PRODUCT_ID'] = df_order['PRODUCT_ID'].astype('str')
df_order['PRODUCT_OPTION_ID'] = df_order['PRODUCT_OPTION_ID'].astype('str')

In [34]:
df_order['PRODUCT_ID'] = df_order['PRODUCT_ID'].str.zfill(4)
df_order['PRODUCT_OPTION_ID'] = df_order['PRODUCT_OPTION_ID'].str.zfill(4)

In [37]:
df_order['PO_ID'] = df_order['PRODUCT_ID']+df_order['PRODUCT_OPTION_ID']

In [38]:
U_nodes = list(df_order['ID'].drop_duplicates())
V_nodes = list(df_order['PO_ID'].drop_duplicates())
edgelist = list(zip(df_order['ID'], df_order['PO_ID']))

In [40]:
b = nx.Graph()
b.add_nodes_from(U_nodes, bipartite=0)
b.add_nodes_from(V_nodes, bipartite=1)

b.add_edges_from(edgelist)

In [41]:
G = bipartite.projected_graph(b, U_nodes)

In [ ]:
pk_internet = {}
num_nodes = len(G.degree())
for i in dict(G.degree()).values():
    if i in pk_internet:
        pk_internet[i] += 1/num_nodes
    else:
        pk_internet[i] = 1/num_nodes
        
x, y = zip(*pk_internet.items())

In [8]:
# db = \
# psycopg2.connect(host='152.70.249.200', dbname='kdsg_sooldamwha',user='kdsg',password='kdsg1234',port=5432)

In [9]:
# cursor=db.cursor()

In [10]:
# def readDB(cursor, schema,table,colum):
#     sql = " SELECT {colum} from {schema}.{table}".format(colum=colum,schema=schema,table=table)
#     try:
# #         self.cursor.execute(sql)
#         cursor.execute(sql)
#         result = cursor.fetchall()
#     except Exception as e :
#         result = (" read DB err",e)
    
#     return result

In [11]:
# print(readDB(cursor = cursor, schema='public',table='orders',colum=['id', 'product_id', 'product_option_id']))

(' read DB err', SyntaxError('syntax error at or near "["\nLINE 1:  SELECT [\'id\', \'product_id\', \'product_option_id\'] from publi...\n                ^\n'))
